In [1]:
'''
This file is used to clean the data and save the cleaned data into a csv file after the EDA process.
Removal of stop words, punctuations, numbers, and special characters.
'''

import pandas as pd
import re
from collections import Counter
import nltk
# remove stop words using nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vedaantibaliga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Data was extracted on April 29, 2023
df = pd.read_csv("ExportedTransactions.csv")

In [3]:
df.head()

,Transaction ID,Posting Date,Effective Date,Transaction Type,Amount,Check Number,Reference Number,Description,Transaction Category,Type,Balance,Memo,Extended Description
0,"20230429 84325 2,454 311,395",4/29/2023,4/29/2023,Credit,24.54,NaN,49460999,Withdrawal Adjustment Debit Card Credit Vo VEN...,NaN,Card,100.70,NaN,Withdrawal Adjustment Debit Card Credit Vo VEN...
1,"20230428 84325 2,048 463,958",4/28/2023,4/28/2023,Debit,-20.48,NaN,49460998,Withdrawal POS # POS TRADER JO TRADER JOES LOS...,NaN,POS,76.16,NaN,Withdrawal POS # POS TRADER JO TRADER JOES LOS...
2,"20230428 84325 6,000 463,870",4/28/2023,4/28/2023,Credit,60.00,NaN,49433161,Deposit Online Banking Transfer From Share,Transfer,NaN,96.64,NaN,Deposit Online Banking Transfer From Share
3,"20230426 84325 1,178 339,604",4/26/2023,4/26/2023,Debit,-11.78,NaN,49372695,Withdrawal Debit Card Debit Card DD DOORDAS...,NaN,Card,36.64,NaN,Withdrawal Debit Card Debit Card DD DOORDAS...
4,"20230425 84325 599 450,226",4/25/2023,4/25/2023,Debit,-5.99,NaN,49372694,Withdrawal POS # POS TRADER JO TRADER JOES ...,NaN,POS,48.42,NaN,Withdrawal POS # POS TRADER JO TRADER JOES ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Transaction ID        875 non-null    object 
 1   Posting Date          875 non-null    object 
 2   Effective Date        875 non-null    object 
 3   Transaction Type      875 non-null    object 
 4   Amount                875 non-null    float64
 5   Check Number          0 non-null      float64
 6   Reference Number      875 non-null    int64  
 7   Description           875 non-null    object 
 8   Transaction Category  305 non-null    object 
 9   Type                  865 non-null    object 
 10  Balance               875 non-null    float64
 11  Memo                  0 non-null      float64
 12  Extended Description  875 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 89.0+ KB


In [5]:
# drop rows with "Description" having the word "libre" (removing fradulent transactions)
df = df.drop(df[df['Description'].str.contains("libre", case=False)].index)
df.reset_index(drop=True, inplace=True)

In [6]:
features_not_needed = ['Transaction ID','Effective Date','Check Number','Reference Number','Type','Memo','Extended Description','Balance']
df = df.drop(features_not_needed,axis=1)
# Convert 'Transaction Category' to type category
df['Transaction Category'] = df['Transaction Category'].astype('category')
# Remove transactions with Credit
df = df[df['Transaction Type'] == 'Debit']
df.reset_index(drop=True, inplace=True)

In [7]:
df.shape

(696, 5)

In [8]:
df['Transaction Category'].value_counts()

Shopping             103
Food & Dining         33
Transfer              10
Personal Care          3
Auto & Transport       1
Business Services      1
Health & Fitness       1
Name: Transaction Category, dtype: int64

In [9]:
df.loc[df['Transaction Category'] == 'Transfer']

,Posting Date,Transaction Type,Amount,Description,Transaction Category
129,1/19/2023,Debit,-120.00,Withdrawal Home Banking Transfer To Share,Transfer
137,1/7/2023,Debit,-118.00,Withdrawal Home Banking Transfer To Share,Transfer
156,11/23/2022,Debit,-2500.00,Withdrawal Home Banking Transfer To Share,Transfer
279,9/12/2022,Debit,-30.00,"Withdrawal Transfer 0025 To CHANDE,VYOMI 04...",Transfer
301,8/23/2022,Debit,-8536.00,Withdrawal Home Banking Transfer To Share,Transfer
313,8/18/2022,Debit,-20200.00,Withdrawal Home Banking Transfer To Share,Transfer
344,7/27/2022,Debit,-20.00,"Withdrawal Transfer 0025 To JOSHI,SAWANEE 6...",Transfer
417,6/5/2022,Debit,-1200.00,Withdrawal Home Banking Transfer To Share,Transfer
523,3/13/2022,Debit,-10200.00,Withdrawal Home Banking Transfer To Share,Transfer
602,12/4/2021,Debit,-16.82,"Withdrawal Transfer 0025 To BANDI,SAIRAM 30...",Transfer


In [10]:
# Remove transactions with "Transfer", we don't need it
df = df[df['Transaction Category'] != 'Transfer']
# Rename some of the categories
df["Transaction Category"] = df["Transaction Category"].cat.rename_categories({"Shopping" : "Shopping & Groceries"})
df["Transaction Category"] = df["Transaction Category"].cat.rename_categories({"Auto & Transport" : "Transportation"})
df["Transaction Category"] = df["Transaction Category"].cat.rename_categories({"Personal Care" : "Recreation"})

In [11]:
# Do a word count for all the words in df['Description']
def word_count(text):
    words_list = []
    for sentence in text:
        words_list += sentence.split()

    word_counts = Counter(words_list)
    # sort the dictionary by value
    word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_dict = dict(word_counts)
    return word_counts_dict

In [12]:
word_count(df['Description']) # same as the barplot in EDA 

{'Debit': 782,
 'Card': 782,
 'Withdrawal': 686,
 'POS': 520,
 'Date': 391,
 'LOS': 253,
 'CA': 244,
 'ANGELES': 218,
 'ST': 207,
 'S': 181,
 '#': 178,
 '3131': 159,
 'TRADER': 136,
 'VENMO': 136,
 'HOOVER': 133,
 'STREET': 101,
 '5812': 91,
 'TARGET': 90,
 'STORE': 80,
 'SOUTH': 77,
 'STE': 75,
 'NY': 72,
 '117': 71,
 'BARROW': 71,
 '4829': 71,
 "JOE'S": 70,
 '#250': 70,
 '4430': 67,
 'T32': 66,
 '19': 58,
 '5814': 56,
 'AVE': 43,
 '4121': 43,
 'W': 43,
 'OF': 42,
 'PARKWAY': 42,
 'UNIV': 41,
 'SO': 41,
 'CAL': 41,
 '7700': 40,
 'EASTPORT': 40,
 'JOES': 34,
 'FIGUEROA': 34,
 'JO': 33,
 'Los': 33,
 '303': 32,
 '*UBER': 31,
 'DD': 27,
 'DOORDASH': 27,
 '2ND': 27,
 'VERMONT': 26,
 'USC': 26,
 'MOBILE': 26,
 'SEATTLE': 25,
 'WA': 25,
 'RALPHS': 24,
 '2600': 24,
 'Angeles': 24,
 '0001': 23,
 'ACH': 22,
 'TYPE:': 22,
 'CO:': 22,
 '5411': 21,
 'WASH': 21,
 'KIOSK': 21,
 '7211': 21,
 '222': 21,
 'MERCHANDSE': 21,
 'MRT': 21,
 'PL': 21,
 'Hoover': 21,
 'PAYMENT': 20,
 '#0294': 20,
 'SAN': 20,


In [13]:
# Cleaning all descriptions and creating word embeddings for data

# Define a function for NLP data cleaning

def clean_text(text):

    # Convert words to lower case.
    text = text.lower()

    # Remove special characters and numbers. This also removes the dates 
    # which are not important in classifying expenses
    text = re.sub(r'[^\w\s]|https?://\S+|www\.\S+|https?:/\S+|[^\x00-\x7F]+|\d+', '', str(text).strip())
  
    # Tokenise 
    text_list = word_tokenize(text)

    # Remove stop words
    text_list = [word for word in text_list if word not in stopwords.words('english')]
    
    result = ' '.join(text_list)
    return result

In [14]:
text_raw = df['Description']
df['Description'] = df['Description'].apply(lambda x: clean_text(x))

In [17]:
# after cleaning
df['Description'].value_counts()

withdrawal debit card debit card venmo barrow street ny date                    70
withdrawal pos pos trader joes south hoover st los angeles ca                   68
withdrawal pos pos target store hoover st ste los angeles ca                    58
withdrawal pos pos trader jo trader joes los angeles ca                         33
withdrawal debit card debit card ralphs vermont ave date                        19
                                                                                ..
withdrawal pos pos marshalls firestone blvd south gate ca                        1
withdrawal pos pos walmart firestone blvd south gate ca                          1
withdrawal debit card debit card sq salt straw w colorado blvd pasade date       1
withdrawal debit card debit card ush parking rc universal city universa date     1
withdrawal pos pos dollar tr venice los angeles ca                               1
Name: Description, Length: 229, dtype: int64

In [18]:
# before training
text_raw.value_counts()

Withdrawal POS #    POS TRADER JO TRADER JOES LOS ANGELES CA                                               29
Withdrawal POS #    POS TRADER JOE'S #250 3131 SOUTH HOOVER ST LOS ANGELES CA                              23
Withdrawal POS # POS TRADER JOE'S #250 3131 SOUTH HOOVER ST LOS ANGELES CA                                 19
Withdrawal POS #    POS TARGET ST 3131 S HOOVE LOS ANGELES CA                                              12
Withdrawal POS #    POS JH BAZAAR LOS ANGELES CA                                                           11
                                                                                                           ..
Withdrawal POS # POS PANDA EXPRESS UNIVERSAL CIT CA                                                         1
Withdrawal Debit Card Debit Card    USC TRANSPORTATION ADM 620 W 35TH ST LOS A Date 09/22/22 5022  4789     1
Withdrawal Debit Card Debit Card SARITAS PUPUSERIA 317 S BROADWAY LOS ANGEL Date 09/22/22 3915  5814        1
Withdrawal

In [20]:
for i in range(0,20):
    print(text_raw.iloc[i])
    print(df['Description'].iloc[i])
    print("\n")

Withdrawal POS # POS TRADER JO TRADER JOES LOS ANGELES CA
withdrawal pos pos trader jo trader joes los angeles ca


Withdrawal Debit Card Debit Card    DD DOORDASH DASHMART 303 2ND STREET 855973 Date 04/25/23 8292  5812
withdrawal debit card debit card dd doordash dashmart nd street date


Withdrawal POS #    POS TRADER JO TRADER JOES LOS ANGELES CA
withdrawal pos pos trader jo trader joes los angeles ca


Withdrawal POS #    POS TARGET ST 3131 S HOOVE LOS ANGELES CA
withdrawal pos pos target st hoove los angeles ca


Withdrawal POS #    POS TRADER JO TRADER JOES LOS ANGELES CA
withdrawal pos pos trader jo trader joes los angeles ca


Withdrawal Debit Card Debit Card    DOMINO'S 7801 2803 S FIGUEROA ST LOS ANGEL Date 04/22/23 0152  5814
withdrawal debit card debit card dominos figueroa st los angel date


Withdrawal Debit Card Debit Card    VENMO 117 BARROW STREET NEW YORK NY    Date 04/23/23 0328  4829
withdrawal debit card debit card venmo barrow street new york ny date


Withdrawal 

In [21]:
word_count(df['Description'])

{'debit': 782,
 'card': 782,
 'withdrawal': 686,
 'pos': 520,
 'date': 391,
 'los': 286,
 'ca': 246,
 'st': 245,
 'angeles': 242,
 'hoover': 154,
 'trader': 136,
 'venmo': 136,
 'street': 106,
 'joes': 104,
 'target': 90,
 'store': 81,
 'south': 81,
 'ste': 78,
 'ny': 72,
 'barrow': 71,
 'ave': 58,
 'figueroa': 46,
 'w': 44,
 'parkway': 42,
 'univ': 41,
 'cal': 41,
 'eastport': 40,
 'doordash': 39,
 'jo': 33,
 'nd': 32,
 'uber': 31,
 'vermont': 28,
 'dd': 27,
 'th': 26,
 'usc': 26,
 'mobile': 26,
 'payment': 25,
 'seattle': 25,
 'wa': 25,
 'ralphs': 24,
 'ach': 22,
 'type': 22,
 'co': 22,
 'san': 22,
 'blvd': 21,
 'wash': 21,
 'kiosk': 21,
 'lyft': 21,
 'merchandse': 21,
 'mrt': 21,
 'pl': 21,
 'dominos': 19,
 'sq': 19,
 'unit': 17,
 'angel': 14,
 'n': 14,
 'jh': 13,
 'bazaar': 13,
 'hoove': 12,
 'name': 12,
 'vedaanti': 12,
 'baliga': 12,
 'berry': 12,
 'wingstop': 12,
 'shein': 10,
 'tst': 10,
 'dieg': 10,
 'trousdale': 10,
 'ride': 9,
 'pkwy': 9,
 'universal': 9,
 'inc': 9,
 'amazon

In [23]:
def replace_words_not_relevant(text):
    # remove words that are not needed or not relevant in the description
    text = text.replace('withdrawal debit card debit card','')
    text = re.sub(r'(\b)pos(\b)', r'\1\2', text)
    text = re.sub(r'(\b)date(\b)', r'\1\2', text)
    text = re.sub(r'(\b)withdrawal(\b)', r'\1\2', text)
    text = re.sub(r'(\b)los(\b)', r'\1\2', text)
    text = re.sub(r'(\b)ca(\b)', r'\1\2', text)
    text = re.sub(r'(\b)angeles(\b)', r'\1\2', text)
    text = re.sub(r'(\b)hoover(\b)', r'\1\2', text)
    text = re.sub(r'(\b)south(\b)', r'\1\2', text)
    text = re.sub(r'(\b)ste(\b)', r'\1\2', text)
    text = re.sub(r'(\b)ny(\b)', r'\1\2', text)
    text = re.sub(r'(\b)barrow(\b)', r'\1\2', text)
    text = re.sub(r'(\b)ave(\b)', r'\1\2', text)
    text = re.sub(r'(\b)figueroa(\b)', r'\1\2', text)
    text = re.sub(r'(\b)w(\b)', r'\1\2', text)
    text = re.sub(r'(\b)nd(\b)', r'\1\2', text)
    text = re.sub(r'(\b)dd(\b)', r'\1\2', text)
    text = re.sub(r'(\b)type(\b)', r'\1\2', text)
    text = re.sub(r'(\b)co(\b)', r'\1\2', text)
    text = re.sub(r'(\b)san(\b)', r'\1\2', text)
    text = re.sub(r'(\b)blvd(\b)', r'\1\2', text)
    text = re.sub(r'(\b)eastport(\b)', r'\1\2', text)
    text = re.sub(r'(\b)st(\b)', r'\1\2', text)
    text = re.sub(r'(\b)th(\b)', r'\1\2', text)
    text = re.sub(r'(\b)wa(\b)', r'\1\2', text)
    text = re.sub(r'(\b)vedaanti baliga(\b)', r'\1\2', text)
    return text


In [24]:
text_before_removal = df['Description']
df['Description'] = df['Description'].apply(lambda x: replace_words_not_relevant(x))

In [25]:
# before cleaning
text_before_removal.value_counts()

withdrawal debit card debit card venmo barrow street ny date                    70
withdrawal pos pos trader joes south hoover st los angeles ca                   68
withdrawal pos pos target store hoover st ste los angeles ca                    58
withdrawal pos pos trader jo trader joes los angeles ca                         33
withdrawal debit card debit card ralphs vermont ave date                        19
                                                                                ..
withdrawal pos pos marshalls firestone blvd south gate ca                        1
withdrawal pos pos walmart firestone blvd south gate ca                          1
withdrawal debit card debit card sq salt straw w colorado blvd pasade date       1
withdrawal debit card debit card ush parking rc universal city universa date     1
withdrawal pos pos dollar tr venice los angeles ca                               1
Name: Description, Length: 229, dtype: int64

In [26]:
# after cleaning
df['Description'].value_counts()

 venmo  street                        70
   trader joes                        68
   target store                       58
   trader jo trader joes              33
 ralphs vermont                       19
                                      ..
 usc transportation adm                1
   panda express universal cit         1
 sq gb coffee grand broadway           1
 sq angels flight deve  hill stre      1
   dollar tr venice                    1
Name: Description, Length: 226, dtype: int64

In [27]:
for i in range(0,20):
    print(text_raw.iloc[i])
    print(df['Description'].iloc[i])
    print("\n")

Withdrawal POS # POS TRADER JO TRADER JOES LOS ANGELES CA
   trader jo trader joes   


Withdrawal Debit Card Debit Card    DD DOORDASH DASHMART 303 2ND STREET 855973 Date 04/25/23 8292  5812
  doordash dashmart  street 


Withdrawal POS #    POS TRADER JO TRADER JOES LOS ANGELES CA
   trader jo trader joes   


Withdrawal POS #    POS TARGET ST 3131 S HOOVE LOS ANGELES CA
   target  hoove   


Withdrawal POS #    POS TRADER JO TRADER JOES LOS ANGELES CA
   trader jo trader joes   


Withdrawal Debit Card Debit Card    DOMINO'S 7801 2803 S FIGUEROA ST LOS ANGEL Date 04/22/23 0152  5814
 dominos    angel 


Withdrawal Debit Card Debit Card    VENMO 117 BARROW STREET NEW YORK NY    Date 04/23/23 0328  4829
 venmo  street new york  


Withdrawal Debit Card Debit Card    CMSVEND*USC 3434 S GRAND AVE STE 12 LOS AN Date 04/21/23 7811  5814
 cmsvendusc grand    


Withdrawal POS #    POS TRADER JO TRADER JOES LOS ANGELES CA
   trader jo trader joes   


Withdrawal POS #    POS TRADER JO TRADE

In [28]:
df.head()

,Posting Date,Transaction Type,Amount,Description,Transaction Category
0,4/28/2023,Debit,-20.48,trader jo trader joes,NaN
1,4/26/2023,Debit,-11.78,doordash dashmart street,NaN
2,4/25/2023,Debit,-5.99,trader jo trader joes,NaN
3,4/25/2023,Debit,-19.00,target hoove,Shopping & Groceries
4,4/24/2023,Debit,-9.05,trader jo trader joes,NaN


In [29]:
df.to_csv('cleaned_transaction_data.csv', index=False)